In [ ]:
import numpy as np
import os
from os import walk
import sys
from time import sleep
import traceback
import joblib
import pandas as pd
import uuid 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM
from sklearn import preprocessing
import distinctipy as dc
import time
import logging
import threading
import sys
import trace
import glob

from PySide6.QtWidgets import QApplication, QMainWindow, QTextEdit,QFileDialog
from PySide6.QtCore import QThread, Signal, QObject
from gui_files.empa_gui import Ui_EmpaGUI
from mod.mod_gui_data_loader_bpp_lines import DataLoaderBPPLines
from mod.mod_gui_feature_extractor_bpp_lines import FeatureExtractorBPPLines
from mod.mod_gui_model_selector_bpp_lines import ModelSelectorBPPLines, PredictorScorerBPPLines
from PySide6.QtWidgets import QMessageBox
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
from matplotlib.figure import Figure
from PySide6.QtWidgets import QDialog, QVBoxLayout
from PySide6.QtWidgets import QMessageBox

#classifiers
from xgboost import XGBClassifier
import torchaudio
import pywt
import ptwt

import SHelpers as shlp

In [ ]:
import torch

class DataPreproc:
    def __init__ (self):
        #SplitEntireSignalIntoSnippets
        self.sign_1=torch.empty
        self.sign_2=torch.empty
        self.sign_3=torch.empty
        self.sign_4=torch.empty      
        self.preproc_1=torch.empty
        self.proceed=False
        #SplitLabPlateSegmentIntoSnips
        self.sign_5=torch.empty   
        self.sign_6=torch.empty
        self.np_signal1=np.empty
        self.np_signal2=np.empty
        #preprocessing
        self.preproc=torch.empty                
        self.preproc_in=torch.empty     
        self.preproc_flat=torch.empty
        self.preproc_final=torch.empty
        #SplitLabPlateAllSegmentIntoSnips
        self.np_signal3=np.empty        
        self.sign_7=torch.empty         
        self.segm_snips_list=[]
        #SplitAllLabPlateSegmentIntoSnips        
        self.sign_8=torch.empty 
        self.sign_9=torch.empty 
        self.segm_labels_list=[]
        self.segm_sign_list=[]
        #SplitAllLabPlateOfAllSegmentsIntoSnips
        self.np_sign_4=torch.empty 
        self.segm_labels_list1=[]
        self.segm_sign_list1=[]       
        self.segm_labels_list2=[]
        self.segm_sign_list2=[]       
        
    #this splits the entire number of segments pf the plate
    def SplitAllPlateSegmentsIntoSnippets(self,plate, 
                                          channs_indx=[], 
                                          snip_size=5,
                                          torch_tensor=True, 
                                          preproc_type="None",         
                                          proc_time=False,
                                         ):
                
        segm_num=len(plate.sigments_sign)
        
        for t in range(0,segm_num):
            self.np_signal3=np.empty
            self.np_signal3=plate.sigments_sign[t]
            self.sign_7=self.SplitEntireSignalIntoSnippets(signal=self.np_signal3, 
                                                      channs_indx=channs_indx, 
                                                      snip_size=snip_size,
                                                      torch_tensor=True, 
                                                      preproc_type=preproc_type,
                                                      proc_time=False)            
            if(torch_tensor==False):
                feat = self.sign_7.cpu().numpy().copy()
                self.segm_snips_list.append(feat)
            else:
                self.segm_snips_list.append(self.sign_7.clone())
        return self.segm_snips_list.copy()

#pass through all plates and 
    def SplitAllLabPlateOfAllSegmentsIntoSnips(self,plates,
                                               snip_size=5,
                                               channs_indx=0,
                                               torch_tensor=False, 
                                               preproc_type="None",                                                
                                              ):
        self.segm_labels_list2=[]
        self.segm_sign_list2=[]       
        
        for pl in plates:
            self.segm_labels_list1=[]
            self.segm_sign_list1=[]       
            self.segm_sign_list1,self.segm_labels_list1=self.SplitAllLabPlateSegmentsIntoSnips(pl,
                                                                                               snip_size=snip_size,
                                                                                               channs_indx=channs_indx,
                                                                                               torch_tensor=torch_tensor, 
                                                                                               preproc_type=preproc_type,  
                                                                                               )
            self.segm_sign_list1,self.segm_labels_list1=self.Helper_FlatListOfLabeledFeat(self.segm_sign_list1,
                                                                                          self.segm_labels_list1
                                                                                         )
            for k in range(0,len(self.segm_sign_list1)):
                self.segm_labels_list2.append(self.segm_labels_list1[k])
                self.segm_sign_list2.append(self.segm_sign_list1[k])      

        return self.segm_sign_list2.copy(),self.segm_labels_list2.copy()

    #this splits all labeled segments into snippets for the given plate
    def SplitAllLabPlateSegmentsIntoSnips(self,plate,
                                         snip_size=5,
                                         channs_indx=-1,
                                         torch_tensor=False, 
                                         preproc_type="None",
                                        ):

        self.segm_labels_list=[]
        self.segm_sign_list=[]
        segm_num=len(plate.sigments_sign)
        
        for t in range(0,segm_num):           
            self.sign_8=torch.empty 
            self.sign_9=torch.empty            
            self.sign_8,self.sign_9=self.SplitLabPlateSegmentIntoSnips(plate,
                                                                       snip_size=snip_size,
                                                                       segm_index=t,
                                                                       channs_indx=channs_indx,
                                                                       torch_tensor=True, 
                                                                       preproc_type=preproc_type,      
                                                                      )
            
            if(self.sign_9 is None or self.sign_8 is None):
                if(torch_tensor==True):
                    self.segm_sign_list.append(torch.empty)
                    self.segm_labels_list.append(torch.empty)
                else:
                    self.segm_sign_list.append(np.empty)
                    self.segm_labels_list.append(np.empty)
            else:
                if(torch_tensor==True):
                    self.segm_sign_list.append(self.sign_8.clone())
                    self.segm_labels_list.append(self.sign_9.clone())
                    
                else:                    
                    self.segm_sign_list.append(self.sign_8.cpu().numpy().copy())
                    self.segm_labels_list.append(self.sign_9.cpu().numpy().copy())
        return self.segm_sign_list.copy(),self.segm_labels_list.copy()
            
    #split obnly labeled data within a segment into snyppets
    def SplitLabPlateSegmentIntoSnips(self,plate,
                                           snip_size=5,
                                           segm_index=0,
                                           channs_indx=-1,
                                           torch_tensor=False, 
                                           preproc_type="None",
                                     ):

        self.np_signal1=np.empty
        self.np_signal2=np.empty
        self.sign_6=torch.empty
        
        lab=[]        
        labeled_data_l=len(plate.sigments_labels[segm_index])    
        self.np_signal1 = np.asarray(plate.sigments_sign[segm_index])        
        for hj in range(0,labeled_data_l):             
            #read labeled data first
            label=plate.sigments_labels[segm_index][hj][2]
            first_el=plate.sigments_labels[segm_index][hj][0]
            last_el=plate.sigments_labels[segm_index][hj][1]
            self.np_signal2=self.np_signal1[:,first_el:last_el].copy()         
            self.sign_5=torch.empty   
            #print(preproc_type)
            #print(self.np_signal2.shape)
            self.sign_5= self.SplitEntireSignalIntoSnippets(  signal=self.np_signal2, 
                                                              channs_indx=channs_indx, 
                                                              snip_size=snip_size,
                                                              torch_tensor=True, 
                                                              preproc_type=preproc_type,
                                                              proc_time=False
                                                            )       
            
            if(self.sign_5 is not None):
                if(self.sign_6==torch.empty):  self.sign_6=self.sign_5.clone()
                else: self.sign_6 = torch.cat([self.sign_6,self.sign_5],dim=0)     
                for k in range(0,self.sign_5.shape[0]):
                    lab.append(label)
            else: pass                
                    
        if(torch_tensor==True):
                    if((self.sign_6!=torch.empty) and (self.sign_6 is not None)):
                        torch_labels_1=torch.tensor(np.asarray(lab))
                        if(torch.cuda.is_available()): torch_labels_1.cuda()                    
                        feat=self.sign_6.clone()
                        return feat, torch_labels_1
                    else: return None,None
        else:    
            if(self.sign_6==torch.empty):
                return None,None
            else:
                feat=self.sign_6.cpu().numpy().copy()
                return feat, np.asarray(lab.copy())               
        
    #split into snippets the given signal
    def SplitEntireSignalIntoSnippets(self,signal=None, 
                                      channs_indx=[], 
                                      snip_size=5,
                                      torch_tensor=True, 
                                      preproc_type="None",
                                      proc_time=False
                                     ):
        self.sign_1=torch.empty
        self.sign_2=torch.empty
        self.sign_3=torch.empty
        self.sign_4=torch.empty        

        if(proc_time==True):
            start_t = time.time()
        self.sign_1=torch.tensor(np.asarray(signal))
        if(torch.cuda.is_available()): self.sign_1.cuda()
        sign_shape=self.sign_1.shape         
        num_snipps=np.round(sign_shape[1]/snip_size)
        for k in range(0,sign_shape[0]):            
            self.proceed = False
            if(type(channs_indx) is not list):
                if(channs_indx == -1): self.proceed=True  
                else: 
                    if(k==channs_indx): self.proceed=True  
            else:
                if(k in channs_indx): self.proceed=True  
                
            if(self.proceed==True):                
                self.sign_2=torch.split(self.sign_1[k],snip_size,dim=0)        
                self.sign_2 = list(self.sign_2)
                while(True): 
                    if(len(self.sign_2)<num_snipps):break                                                      
                    del self.sign_2[-1]
                self.sign_3=torch.stack(self.sign_2, dim=0) 
                #preprocessing
                if(preproc_type=="None" or  preproc_type=="" or preproc_type==" "): pass
                else: self.sign_3=self.DataPreprocessing(self.sign_3,preproc_type=preproc_type)
                #assign further
                if(self.sign_4 == torch.empty): self.sign_4 = self.sign_3 
                else: self.sign_4 = torch.cat([self.sign_4,self.sign_3],dim=1)#stack((self.sign_4,self.sign_3),dim=1) 
        if(proc_time==True):
            end_t = time.time()
            print("Feat. extr. time(s): "+str(end_t - start_t))
        if(torch_tensor==True):
            return self.sign_4.clone()
        else:
            return self.sign_4.cpu().numpy().copy()      

    #------------------obtain spectrograms-------------------------
    #------------------as a preprocessing step---------------------
    #--------------------------------------------------------------
    def DataPreprocessing(self,signal,preproc_type="torch_MEL"):
        if(torch.cuda.is_available()):
            if(signal.is_cuda==False):
                signal.cuda()
                
        self.preproc=torch.empty
        self.preproc_final=torch.empty

        shp=signal.shape
        
        if(preproc_type=="torch_MEL"):
            #parameters
            n_fft = 128         #256       #Toni_MEL_nfft
            win_length = None
            hop_length = 2024   #2024  #Toni_MEL_Samp_hop_length
            n_mels = 512 #512   #self.Toni_MEL_Samp_n_mels
            sample_rate = 1000000 #40000 #self.Toni_MEL_samp_rate
            #MEL decomposition            
            for l in range(0,shp[0]):
                self.preproc_in=torch.empty
                self.preproc_in=signal[l,:].reshape((1, -1))                
                #https://pytorch.org/audio/stable/tutorials/audio_feature_extractions_tutorial.html#sphx-glr-tutorials-audio-feature-extractions-tutorial-py
                MEL_transoform = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate,
                                                                                n_fft=n_fft,
                                                                                win_length=win_length,
                                                                                hop_length=hop_length,
                                                                                center=True,
                                                                                pad_mode="reflect",
                                                                                power=2.0,
                                                                                norm="slaney",
                                                                                n_mels=n_mels,
                                                                                mel_scale="htk",).to(torch.double)   
                
                self.preproc = MEL_transoform(self.preproc_in)                
                self.preproc_flat = self.preproc.reshape(-1)    
                self.preproc_flat=self.preproc_flat.unsqueeze(0)                
                if(self.preproc_final==torch.empty): self.preproc_final = self.preproc_flat
                else:   self.preproc_final=torch.cat([self.preproc_final,self.preproc_flat],dim=0)                    
            return self.preproc_final.clone()
        #fft
        if(preproc_type=="torch_FFT"):
            #https://www.kaggle.com/code/yassinealouini/signal-processing-theory-and-practice-with-pytorch
            for l in range(0,shp[0]):
                self.preproc_in=torch.empty
                self.preproc_in=signal[l,:].reshape((1, -1))                
                self.preproc=torch.fft.fft(self.preproc_in)
                shp_proc=self.preproc.shape
                cut_half=int(shp_proc[1]/2)
                self.preproc=self.preproc[:,0:]
                if(self.preproc_final==torch.empty): self.preproc_final = self.preproc
                else:   self.preproc_final=torch.cat([self.preproc_final,self.preproc],dim=0)                       
            return self.preproc_final.clone()
        #mfcc spectrogram
        if(preproc_type=="torch_MFCC"):
            #parameters
            n_mfcc=13
            sample_rate = 1000000 #40000 
            
            #MEL decomposition            
            for l in range(0,shp[0]):
                self.preproc_in=torch.empty
                self.preproc_in=signal[l,:].reshape((1, -1))                
                #https://pytorch.org/audio/stable/tutorials/audio_feature_extractions_tutorial.html#sphx-glr-tutorials-audio-feature-extractions-tutorial-py
                MFCC_transoform = torchaudio.transforms.MFCC(sample_rate=sample_rate,
                                                            n_mfcc=n_mfcc,
                                                            melkwargs={"n_fft": 50, "hop_length": 50, "n_mels": 23, "center": False},
                                                           ).to(torch.double)   
                
                self.preproc = MFCC_transoform(self.preproc_in)                
                self.preproc_flat = self.preproc.reshape(-1)    
                self.preproc_flat=self.preproc_flat.unsqueeze(0)                
                if(self.preproc_final==torch.empty): self.preproc_final = self.preproc_flat
                else:   self.preproc_final=torch.cat([self.preproc_final,self.preproc_flat],dim=0)                    
            return self.preproc_final .clone()

        #widow fft spectrogram
        if(preproc_type=="torch_WFFT"):
            #parameters
            n_fft=50
            win_len=None
            hop_len=None
            power=2.0
            
            #MEL decomposition            
            for l in range(0,shp[0]):
                self.preproc_in=torch.empty
                self.preproc_in=signal[l,:].reshape((1, -1))                
                #https://pytorch.org/audio/stable/tutorials/audio_feature_extractions_tutorial.html#sphx-glr-tutorials-audio-feature-extractions-tutorial-py
                WFFT_transoform = torchaudio.transforms.Spectrogram(n_fft=n_fft,
                                                                    win_length=win_len,
                                                                    hop_length=hop_len,
                                                                    center=True,
                                                                    pad_mode="reflect",
                                                                    power=power,
                                                                    ).to(torch.double)              
                self.preproc = WFFT_transoform(self.preproc_in)                
                self.preproc_flat = self.preproc.reshape(-1)    
                self.preproc_flat=self.preproc_flat.unsqueeze(0)                
                if(self.preproc_final==torch.empty): self.preproc_final = self.preproc_flat
                else:   self.preproc_final=torch.cat([self.preproc_final,self.preproc_flat],dim=0)                    
            return self.preproc_final .clone()
            
        #Griffin-Lim transform
        if(preproc_type=="torch_GrifLim"):
            #parameters
            n_fft=64 #should be order of two
                        
            #MEL decomposition            
            for l in range(0,shp[0]):
                self.preproc_in=torch.empty
                self.preproc_in=signal[l,:].reshape((1, -1))       
                #https://pytorch.org/audio/stable/tutorials/audio_feature_extractions_tutorial.html#sphx-glr-tutorials-audio-feature-extractions-tutorial-py
                GL_transoform = torchaudio.transforms.GriffinLim(n_fft=n_fft).to(torch.double)              
                self.preproc = GL_transoform(self.preproc_in)                
                self.preproc_flat = self.preproc.reshape(-1)    
                self.preproc_flat=self.preproc_flat.unsqueeze(0)                
                if(self.preproc_final==torch.empty): self.preproc_final = self.preproc_flat
                else:   self.preproc_final=torch.cat([self.preproc_final,self.preproc_flat],dim=0)                    
            return self.preproc_final.clone()

        #Griffin-Lim transform
        if(preproc_type=="torch_LFCC"):
            #parameters            
            sample_rate = 1000000
            n_lfcc=13
                        
            #MEL decomposition            
            for l in range(0,shp[0]):
                self.preproc_in=torch.empty
                self.preproc_in=signal[l,:].reshape((1, -1))       
                #https://pytorch.org/audio/stable/tutorials/audio_feature_extractions_tutorial.html#sphx-glr-tutorials-audio-feature-extractions-tutorial-py
                LFCC_transoform = torchaudio.transforms.LFCC(sample_rate=sample_rate,
                                                             n_lfcc=n_lfcc,
                                                             speckwargs={"n_fft": 40, "hop_length": 30, "center": False},
                                                            ).to(torch.double)              
                self.preproc = LFCC_transoform(self.preproc_in)                
                self.preproc_flat = self.preproc.reshape(-1)    
                self.preproc_flat=self.preproc_flat.unsqueeze(0)                
                if(self.preproc_final==torch.empty): self.preproc_final = self.preproc_flat
                else:   self.preproc_final=torch.cat([self.preproc_final,self.preproc_flat],dim=0)                    
            return self.preproc_final.clone()

        #Spec centroid
        if(preproc_type=="torch_SpecCentr"):
            #parameters            
            sample_rate = 1000000
            n_fft = 50
                        
            #MEL decomposition            
            for l in range(0,shp[0]):
                self.preproc_in=torch.empty
                self.preproc_in=signal[l,:].reshape((1, -1))       
                #https://pytorch.org/audio/stable/tutorials/audio_feature_extractions_tutorial.html#sphx-glr-tutorials-audio-feature-extractions-tutorial-py
                SC_transoform = torchaudio.transforms.SpectralCentroid(sample_rate=sample_rate,
                                                                       n_fft=n_fft,
                                                                      ).to(torch.double)              
                self.preproc = SC_transoform(self.preproc_in)                
                self.preproc_flat = self.preproc.reshape(-1)    
                self.preproc_flat=self.preproc_flat.unsqueeze(0)                
                if(self.preproc_final==torch.empty): self.preproc_final = self.preproc_flat
                else:   self.preproc_final=torch.cat([self.preproc_final,self.preproc_flat],dim=0)                    
            return self.preproc_final.clone()

    #helper functions
    def Helper_FlatListOfLabeledFeat(self,feat,labels):
        feat_new=[]
        labs=[]
        h_l=len(feat)
        for i in range(0,h_l):      
            if(feat[i] is not None): 
                shp = np.shape(feat[i])       
                if(len(shp)>=1):
                    for j in range(0,shp[0]):
                        feat_new.append(feat[i][j][:])
                        labs.append(labels[i][j])
        return feat_new,labs
            
#dp=DataPreproc()
#fd= dp.SplitEntireSignalIntoSnippets(signal=PLATES_ARRAY[0].sigments_sign[0],channs_indx=0,torch_tensor=False,snip_size=1000)
#fd,labs= dp.SplitLabPlateSegmentIntoSnips(PLATES_ARRAY[0],snip_size=5,segm_index=0,channs_indx=[0])

In [ ]:
dp=DataPreproc()
#fd= dp.SplitAllPlateSegmentsIntoSnippets(PLATES_ARRAY[0],channs_indx=[1],torch_tensor=False,snip_size=100)
#feat,labels= dp.SplitAllLabPlateSegmentsIntoSnips(PLATES_ARRAY[0],channs_indx=[1],torch_tensor=False,snip_size=100)
#f,l=dp.Helper_FlatListOfLabeledFeat(feat,labels)
#print(np.shape(np.asarray(f)))
feat,labels= dp.SplitAllLabPlateOfAllSegmentsIntoSnips(PLATES_ARRAY,channs_indx=[1],torch_tensor=False,snip_size=100)

In [ ]:
dp=DataPreproc()
fd= dp.SplitEntireSignalIntoSnippets(signal=PLATES_ARRAY[0].sigments_sign[0],
                                     channs_indx=[0,1],
                                     torch_tensor=False,
                                     snip_size=10,
                                     preproc_type="tocrh_DWT")